# Diorama single L1 TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data of Diorama.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import pandas as pd
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
from datetime import datetime
today = datetime.today()
# dd/mm/YY/Hh/Mm
d4 = today.strftime("%d-%b-%Y-%Hh:%Mm")

### Auxiliary functions

In [3]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing Anitapolis TFA data

In [4]:
data = pd.read_csv('diorama_decimated_tfa_residual.txt', skipinitialspace=True, delim_whitespace=True)

In [5]:
xp = data['X'].get_values()
yp = data['Y'].get_values()
zp = data['GPSALT'].get_values()
alt = data['ALTURA'].get_values()
dobs = data['mag_res2'].get_values()

### Parameters of the initial model

In [6]:
M = 20 # number of vertices per prism
L = 6 # number of prisms
P = L*(M+2) + 1 # number of parameters

# depth to the top, thickness, origin, magnetization and radius
incs = -46.
decs = 24.
intensity = 18.
z0 = 250.
dz = 350.
r = 1200.
x0 = 8212500.
y0 = 478200.

# main field
inc, dec = [-19.5, -18.]

model0, m0 = mfun.initial_cylinder(M, L, x0,
        y0, z0, dz, r, inc, dec, incs, decs, intensity)

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

### Outcropping parameters

In [7]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Limits

In [8]:
# limits for parameters in meters
rmin = 10.
rmax = 1500.
x0min = 8212500.
x0max = 8213500.
y0min = 478000.
y0max = 479000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Derivatives

In [9]:
# variation for derivatives
deltax = 0.01*np.max(100.)
deltay = 0.01*np.max(100.)
deltar = 0.01*np.max(100.)
deltaz = 0.01*np.max(100.)

delta = np.array([deltax, deltay, deltar, deltaz])

### Regularization parameters

In [10]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-5  # adjacent radial distances within each prism
a2 = 1.0e-5 # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4    # vertically adjacent origins
a6 = 1.0e-6   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-6     # zero order Tikhonov on thickness of each prism

alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

### Folder to save the results

In [15]:
foldername = 'test'

### Iterations and stop criterion

In [12]:
itmax = 30       # maximum iteration
itmax_marq = 10  # maximum iteration of Marquardt's loop
tol = 1.0e-4     # stop criterion

### Inversion

In [14]:
d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, inc, dec,
    model0[0].props, alpha, z0, dz, 1
)

it:  0   it_marq:  0   lambda: 1e+01   init obj.: 1.53975e+02  fin obj.: 1.30342e+02
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 1.30342e+02  fin obj.: 1.43992e+02
it:  1   it_marq:  1   lambda: 1e+01   init obj.: 1.30342e+02  fin obj.: 1.44092e+02
it:  1   it_marq:  2   lambda: 1e+02   init obj.: 1.30342e+02  fin obj.: 1.30099e+02
it:  2   it_marq:  0   lambda: 1e+01   init obj.: 1.30099e+02  fin obj.: 1.28423e+02
it:  3   it_marq:  0   lambda: 1e+00   init obj.: 1.28423e+02  fin obj.: 1.23669e+02
it:  4   it_marq:  0   lambda: 1e-01   init obj.: 1.23669e+02  fin obj.: 1.17109e+02
it:  5   it_marq:  0   lambda: 1e-02   init obj.: 1.17109e+02  fin obj.: 1.50922e+02
it:  5   it_marq:  1   lambda: 1e-01   init obj.: 1.17109e+02  fin obj.: 1.51054e+02
it:  5   it_marq:  2   lambda: 1e+00   init obj.: 1.17109e+02  fin obj.: 1.49888e+02
it:  5   it_marq:  3   lambda: 1e+01   init obj.: 1.17109e+02  fin obj.: 1.49126e+02
it:  5   it_marq:  4   lambda: 1e+02   init obj.: 1.17109e+02  fi

# Results

In [16]:
# output of inversion
inversion = dict()

inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['origin'] = [x0, y0]
inversion['observed_data'] = dobs
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = np.array([a1, a2, a3, a4, a5, a6, a7])
inversion['tol'] = tol
inversion['main_field'] = [-21.5, -18.7]
inversion['data_fit'] = d_fit
inversion['estimate'] = m_est
inversion['prisms'] = model_est
inversion['estimated_models'] = model_list
inversion['objective'] = phi_list
inversion['residual'] = dobs - d_fit
inversion['residual_list'] = res_list

### Saving results

In [17]:
if foldername == '':
    mypath = 'l1-tfa-inversion/single-'+d4 #default folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
else:
    mypath = 'l1-tfa-inversion/single-'+foldername #defined folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

In [18]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)